<a href="https://colab.research.google.com/github/micah-shull/RAG-LangChain/blob/main/LC_011_RAG_CustServiceBot_PromptOptimzation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ✅ Changes to Implement

### 🔧 1. **Increase Retriever Depth**

* Change `k = 2` → `k = 4`
  → To ensure retrieval of deeper content, especially pricing and onboarding details near the end of docs.

---

### ✍️ 2. **Update Source Documents**

* **Moved important content up** (e.g., pricing, free assessment)
* **Renamed vague headers** (e.g., `# Summary` → `# Cashflow4cast reduces forecasting errors by`)
* **Added new structured section** for data privacy and onboarding steps

---

### 📄 3. **Enhanced Prompt with Few-Shot Examples**

* Added 6 tailored examples:

  * What does Cashflow4cast do?
  * How does it work?
  * What does it cost?
  * How often are forecasts updated?
  * What economic indicators do you use?
  * How do you test if my data is a good fit?

---

### 🧠 4. **Improve Model Guidance**

* Few-shot examples now:

  * Emphasize clarity, tone, structure
  * Include sensitive topics like **data privacy** and **trust-building**
  * Highlight **full pricing tiers** and **onboarding steps**

---

These changes are designed to improve **retrieval coverage, response completeness, and customer trust signals** in the chatbot response.




## Pip Intsall


In [1]:
!pip install --upgrade --quiet \
    langchain \
    langchain-huggingface \
    langchain-openai \
    langchain-community \
    chromadb \
    python-dotenv \
    transformers \
    accelerate \
    sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 76.5 MB/s eta 0:

## Load Libs

In [2]:
# 🌿 Environment setup
import os                                 # File paths and OS interaction
from dotenv import load_dotenv            # Load environment variables from .env file
import langchain; print(langchain.__version__)  # Check LangChain version

# 📄 Document loading and preprocessing
from langchain_core.documents import Document                   # Base document type
from langchain_community.document_loaders import TextLoader     # Loads plain text files
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Splits long docs into smaller chunks

# 🔢 Embeddings + vector storage
from langchain_huggingface import HuggingFaceEmbeddings         # HuggingFace embedding model
from langchain.vectorstores import Chroma                       # Persistent vector DB (Chroma)

# 💬 Prompting + output
from langchain_core.prompts import ChatPromptTemplate           # Chat-style prompt templates
from langchain_core.output_parsers import StrOutputParser       # Converts model output to string

# 🔗 Chains / pipelines
from langchain_core.runnables import Runnable, RunnableLambda   # Compose custom pipelines

# 🧠 (Optional) Hugging Face LLM client setup
# from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace  # For HF inference API

# 🧾 Pretty printing
import textwrap                         # Format long strings for printing
from pprint import pprint               # Nicely format nested data structures

0.3.25


## Set Params


In [9]:
from langchain_openai.chat_models.base import ChatOpenAI

# SET MODEL PARAMS
EMBED_MODEL = "all-MiniLM-L6-v2"
CHUNK_SIZE = 250
CHUNK_OVERLAP = 50
K = 4

# Load token from .env.
load_dotenv("/content/API_KEYS.env", override=True)

LLM_MODEL = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # or "gpt-4-turbo" if budget allows
    temperature=0.3,  # factual, deterministic
    max_tokens=500  # adjust based on response length
)


## Load Docs - Clean & Chunk

In [6]:
# Path to your documents
docs_path = "/content/sample_data/CFFC_RAG_DOCS"

# Step 1: Load all .txt files in the folder
raw_documents = []
for filename in os.listdir(docs_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(docs_path, filename)
        loader = TextLoader(file_path, encoding="utf-8")
        docs = loader.load()
        for doc in docs:
            doc.metadata["source"] = filename # reserve source filenames for tracing
        raw_documents.extend(docs)

print(f"Loaded {len(raw_documents)} documents.")

# Step 2 (optional): Clean up newlines and extra whitespace
def clean_doc(doc: Document) -> Document:
    cleaned = " ".join(doc.page_content.split())  # Removes newlines & extra spaces
    return Document(page_content=cleaned, metadata=doc.metadata)

cleaned_documents = [clean_doc(doc) for doc in raw_documents]

# Step 3: Split documents into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

chunked_documents = splitter.split_documents(cleaned_documents)

# Check average chunk size (debug aid)
avg_length = sum(len(doc.page_content) for doc in chunked_documents) / len(chunked_documents)
print(f"📏 Average chunk length: {avg_length:.1f} characters")


print(f"Split into {len(chunked_documents)} total chunks.")

# Preview the first 5 chunks
print(f"Showing first 5 of {len(chunked_documents)} chunks:\n")

for i, doc in enumerate(chunked_documents[:5]):
    print(f"--- Chunk {i+1} ---")
    print(f"Source: {doc.metadata.get('source', 'N/A')}\n")
    print(textwrap.fill(doc.page_content[:500], width=100))  # limit preview to 500 characters
    print("\n")

Loaded 5 documents.
📏 Average chunk length: 240.8 characters
Split into 62 total chunks.
Showing first 5 of 62 chunks:

--- Chunk 1 ---
Source: CFFC_Features_UseCases.txt

# 📊 Forecasting You Can Trust in Uncertain Times We help mid-sized businesses stay ahead of sales
volatility and protect their bottom line by cutting forecast errors in half — using powerful machine
learning models. --- ## 🚀 Key Benefits at a Glance


--- Chunk 2 ---
Source: CFFC_Features_UseCases.txt

models. --- ## 🚀 Key Benefits at a Glance - Cut forecasting errors by **50% or more** - Detect
changes in demand **before they impact cash flow** - Respond to opportunities with **confidence and
speed** - Replace spreadsheets with **smarter,


--- Chunk 3 ---
Source: CFFC_Features_UseCases.txt

speed** - Replace spreadsheets with **smarter, adaptive forecasting** --- ## ⚠️ 1. Risk: Can I Avoid
Costly Surprises? **Traditional Tools (Excel, QuickBooks):** - Assume tomorrow looks like yesterday
- Miss sudden drops in dema

## Embeddings

In [7]:
# Step 1: Set up Hugging Face embedding model
embedding_model = HuggingFaceEmbeddings(model_name=EMBED_MODEL)

# Step 2: Set up Chroma with persistence
persist_dir = "chroma_db"

vectorstore = Chroma.from_documents(
    documents=chunked_documents,
    embedding=embedding_model,
    persist_directory=persist_dir
)

print(f"✅ Stored {len(chunked_documents)} chunks in Chroma at '{persist_dir}'")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Stored 62 chunks in Chroma at 'chroma_db'


## Retriever & Prompt

In [13]:
retriever = vectorstore.as_retriever(search_kwargs={"k": K})

# promtp template
prompt_template = ChatPromptTemplate.from_template("""
You are a customer support assistant for **Cashflow4cast**, a platform that helps businesses improve cash flow forecasting using machine learning and economic indicators.

Answer the user's question using only the information provided in the business documents below.

Instructions:
- If the answer is not in the context, respond with: "I'm sorry, I don't have that information based on the current documentation."
- Be clear, concise, and professional, but use natural language — answer like you're explaining to a business owner, not a computer.
- Use bullet points or short lists where appropriate (e.g., pricing, features).
- Do not include personal opinions or make assumptions.
- Do not mention services, features, or pricing unless explicitly included in the provided context.

---

### Example 1

**Question**: What does Cashflow4cast actually do?

**Answer**:
- Cashflow4cast helps small and mid-sized businesses improve financial forecasting accuracy
- It provides more accurate forecasting than spreadsheets for all types of businesses
- Cashflow4cast unlocks savings, enables smarter planning, and offers peace of mind for business owners
- The platform delivers consistent forecasting accuracy across multiple stores, ensuring reliable and repeatable performance

---

### Example 2

**Question**: How does this forecasting service work?

**Answer**:
- Adapts dynamically as your business grows, allowing you to confidently expand without overwhelming your systems
- Focuses on 30-day forecasts to deliver more value compared to traditional tools that may lose reliability fast
- Provides accurate forecasts with less hassle
- Offers a 50%+ error reduction in forecasting
- Enables smart planning for inventory, staffing, and cash flow
- Acts as a partner that works with your data, not against it

---

### Example 3

**Question**: What does it cost?

**Answer**:
Cashflow4cast offers three pricing tiers:

- **Basic Plan – $199/month**
  Best for small businesses new to forecasting.
  Includes: monthly forecasts, category-level insights, and Excel/CSV reports.

- **Advanced Plan – $499/month**
  Best for growing teams needing deeper insights.
  Includes: weekly forecasts, SKU-level granularity, and promotion tracking.

- **Premium Plan – $999/month**
  Best for businesses needing real-time support.
  Includes: daily forecasts, dashboards, economic indicators, and 1-on-1 consulting.

---

### Example 4

**Question**: How often are forecasts updated?

**Answer**:
Forecast frequency depends on your selected plan:

- **Basic Plan** → Monthly forecasts
- **Advanced Plan** → Weekly forecasts
- **Premium Plan** → Daily forecasts with real-time dashboards

Each tier is designed to match the level of insight and responsiveness your business needs.

---

### Example 5

**Question**: What economic indicators do you use?

**Answer**:
Cashflow4cast incorporates a range of economic indicators to improve forecasting accuracy:

- **Consumer Price Index (CPI)** – Measures inflation and cost pressures
- **Consumer Confidence Index (CCI)** – Reflects consumer optimism and willingness to spend
- **Retail Sales** – Shows real-time shifts in consumer demand
- **Unemployment Rate** – Influences labor availability and income levels
- **GDP Growth** – Signals the overall health of the economy

These indicators help anticipate demand shifts, guide pricing, and support smarter planning.

---

### Example 6

**Question**: How do you test if my data is a good fit?

**Answer**:
To ensure your data is a good fit for machine learning forecasting, we offer a quick, low-risk pilot:

- **Step 1**: You send a small sample of your recent sales history (no installation required)
- **Step 2**: We train a machine learning model and generate forecasts for the next 2–3 months
- **Step 3**: We compare those results to traditional spreadsheet forecasts, side-by-side
- **Step 4**: You review the results and decide if it's the right fit

> **Your data stays private and secure**, and there’s no commitment to continue after the test.

---

### Example 7

**Question**: How long does it take to get started?

**Answer**:
Setup is quick and straightforward:

- Once you share a sample of your sales history, we’ll begin reviewing your data and building the model
- You’ll receive preliminary test forecasts in about a week
- No software installation or technical onboarding is required
- You'll get a side-by-side comparison with traditional methods and clear next-step recommendations

---

### Example 8

**Question**: Is this based on my actual sales data or just economic trends?

**Answer**:
Yes — Cashflow4cast is trained using your actual sales history.

- During onboarding, you share a sample of your recent sales data
- Our machine learning model is then trained to learn your specific business patterns
- Economic indicators like inflation and consumer confidence are layered in to enhance the forecast

This ensures your forecasts are tailored to your business, not just broad market trends.


Context:
{context}

Question:
{question}

Answer:
""")


## RAG Chain

In [11]:
# Define RAG chain
rag_chain = (
    RunnableLambda(lambda d: {
        "question": d["question"],
        "docs": retriever.invoke(d["question"])
    })
    # | RunnableLambda(lambda d: {
    #     "context": "\n\n".join([doc.page_content for doc in d["docs"]]),
    #     "question": d["question"]
    # })

    | RunnableLambda(lambda d: {
    "context": "\n\n".join(
        [f"[Source: {doc.metadata.get('source', 'N/A')}]\n{doc.page_content}" for doc in d["docs"]]
    ),
    "question": d["question"]
    })

    | prompt_template
    | LLM_MODEL
    | StrOutputParser()
)

# Invoke RAG
response = rag_chain.invoke({
    "question": "What are the recent economic indicators that affect local businesses?"
})

# Print response nicely
import textwrap
print("\n" + textwrap.fill(response, width=100))



- **Consumer Price Index (CPI)**   - Tracks average price changes   - Influences consumer behavior
and pricing decisions  - **Consumer Confidence Index (CCI)**   - Reflects consumer optimism and
willingness to spend   - Impacts sales rhythms and demand levels  - **Retail Sales**   - Shows real-
time shifts in consumer demand   - Can affect local businesses through changes in national retail
spending


# TEST QUESTIONS

In [12]:
# Define test questions
test_questions = [
    "What does Cashflow4cast actually do?",
    "How does this forecasting service work?",
    "Is this based on my actual sales data or general trends?"
]

# Function to run and display answers
def run_rag_tests(questions, rag_chain, wrap_width=100):
    for i, question in enumerate(questions, 1):
        print(f"\n🔹 Test {i}: {question}\n{'-' * wrap_width}")
        response = rag_chain.invoke({"question": question})
        print(textwrap.fill(response, width=wrap_width))
        print("\n" + "=" * wrap_width)

# Run the first three tests
run_rag_tests(test_questions, rag_chain)



🔹 Test 1: What does Cashflow4cast actually do?
----------------------------------------------------------------------------------------------------
- Cashflow4cast helps small and mid-sized businesses improve financial forecasting accuracy - It
uses machine learning and economic indicators to provide more accurate forecasting than spreadsheets
for all types of businesses - The platform unlocks savings, enables smarter planning, and offers
peace of mind for business owners - Cashflow4cast delivers consistent forecasting accuracy across
multiple stores, ensuring reliable and repeatable performance


🔹 Test 2: How does this forecasting service work?
----------------------------------------------------------------------------------------------------
- Adapts dynamically as your business grows, allowing you to confidently expand without overwhelming
your systems - Focuses on 30-day forecasts to deliver more value compared to traditional tools that
may lose reliability fast - Provides accur

### TEST 1 - 🧠 General Understanding

In [14]:
test_general = [
    "What does Cashflow4cast actually do?",
    "How does this forecasting service work?",
    "Is this based on my actual sales data or general trends?",
    "How is this different from Excel or QuickBooks forecasts?",
    "Can I trust forecasts during unstable economic conditions?"
]

# Run the first three tests
run_rag_tests(test_general, rag_chain)


🔹 Test 1: What does Cashflow4cast actually do?
----------------------------------------------------------------------------------------------------
- Cashflow4cast helps small and mid-sized businesses improve financial forecasting accuracy - It
reduces forecasting errors using a machine learning model trained on actual sales history - The
platform provides more accurate and dynamic forecasts compared to traditional tools like Excel or
Prophet - Cashflow4cast unlocks savings, enables smarter planning, and offers peace of mind for
business owners


🔹 Test 2: How does this forecasting service work?
----------------------------------------------------------------------------------------------------
- Adapts dynamically as your business grows, allowing you to expand confidently without overwhelming
your systems - Focuses on 30-day forecasts to deliver more value compared to traditional tools that
may lose reliability fast - Provides accurate forecasts with less hassle - Offers a 50%+ error

### TEST 2 - 💰 Pricing & Plans

In [15]:
test_pricing = [
    "What does it cost?",
    "What’s included in each plan?",
    "Is there a free trial?",
    "Do you offer consulting or custom analysis?",
    "What’s the difference between the Basic and Premium plans?"
]

# Run the first three tests
run_rag_tests(test_pricing, rag_chain)


🔹 Test 1: What does it cost?
----------------------------------------------------------------------------------------------------
- Cashflow4cast offers three pricing tiers:   - **Basic Plan – $199/month**     - Best for small
businesses new to forecasting.     - Includes: monthly forecasts, category-level insights, and
Excel/CSV reports.   - **Advanced Plan – $499/month**     - Best for growing teams needing deeper
insights.     - Includes: weekly forecasts, SKU-level granularity, and promotion tracking.   -
**Premium Plan – $999/month**     - Best for businesses needing real-time support.     - Includes:
daily forecasts, dashboards, economic indicators, and 1-on-1 consulting.


🔹 Test 2: What’s included in each plan?
----------------------------------------------------------------------------------------------------
- **Basic Plan – $199/month**   - Best for small businesses new to forecasting   - Includes: monthly
forecasts, category-level insights, and Excel/CSV reports  - **Advan

### TEST 3 - 📈 Forecasting & Accuracy

In [16]:
test_forecasting = [
    "How accurate are your forecasts?",
    "How do you reduce forecasting errors?",
    "Can you forecast by product or store?",
    "How do you handle promotions or seasonal trends?",
    "How often are forecasts updated?"
]

# Run the first three tests
run_rag_tests(test_forecasting, rag_chain)


🔹 Test 1: How accurate are your forecasts?
----------------------------------------------------------------------------------------------------
- Cashflow4cast significantly reduces forecasting errors, resulting in more accurate forecasts -
Traditional forecasting methods often have large error margins, while Cashflow4cast offers a more
precise prediction - The platform helps business owners plan with greater confidence and clarity


🔹 Test 2: How do you reduce forecasting errors?
----------------------------------------------------------------------------------------------------
- Cashflow4cast significantly reduces forecasting errors compared to traditional methods - The
platform offers more accurate forecasting than spreadsheets, which often result in large error
margins - By using machine learning and economic indicators, Cashflow4cast helps business owners
plan with greater confidence and clarity


🔹 Test 3: Can you forecast by product or store?
----------------------------------

### TEST 4 - 🌐 Economic Indicators

In [17]:
test_economics = [
    "What economic indicators do you use?",
    "How does inflation affect my business?",
    "What does the Consumer Confidence Index mean for sales?",
    "Do national trends really affect local businesses?",
    "What should I be watching in the economy right now?"
]

# Run the first three tests
run_rag_tests(test_economics, rag_chain)


🔹 Test 1: What economic indicators do you use?
----------------------------------------------------------------------------------------------------
- Cashflow4cast incorporates the following economic indicators to improve forecasting accuracy:   -
Consumer Price Index (CPI) – Measures inflation and cost pressures   - Consumer Confidence Index
(CCI) – Reflects consumer optimism and willingness to spend   - Retail Sales – Shows real-time
shifts in consumer demand   - Unemployment Rate – Influences labor availability and income levels
- GDP Growth – Signals the overall health of the economy


🔹 Test 2: How does inflation affect my business?
----------------------------------------------------------------------------------------------------
- Inflation can impact your business by affecting customer purchasing power - Rising inflation may
lead to customers having less discretionary income - Higher prices for supplies, shipping, and labor
can reduce your profit margins - Business owners may

### TEST 5 - 👣 Onboarding & Setup

In [18]:
test_onboarding = [
    "What’s the first step if I want to try this?",
    "How do you test if my data is a good fit?",
    "Do I need to install anything?",
    "How long does setup take?"
]

# Run the first three tests
run_rag_tests(test_onboarding, rag_chain)


🔹 Test 1: What’s the first step if I want to try this?
----------------------------------------------------------------------------------------------------
The first step to try Cashflow4cast is to schedule a free consultation to assess if your data is a
good fit for machine learning forecasting.


🔹 Test 2: How do you test if my data is a good fit?
----------------------------------------------------------------------------------------------------
- Cashflow4cast offers a free fit assessment to determine if your data is suitable for machine
learning forecasting - The process involves sending a sample of your recent sales history for
analysis - A machine learning model is trained on your data to generate forecasts for the next 2-3
months - Results are compared with traditional spreadsheet forecasts to evaluate accuracy - Your
data remains private and secure throughout the assessment - There is no commitment to continue after
the test, ensuring a low-risk evaluation process


🔹 Test 3:

### TEST 6 - 🚫 Fallback Testing (Out-of-Scope)



In [19]:
test_fallbacks = [
    "Do you support inventory tracking and reordering?",
    "Can I use this to file my taxes?",
    "How do I open a store in another state?"
]

# Run the first three tests
run_rag_tests(test_fallbacks, rag_chain)


🔹 Test 1: Do you support inventory tracking and reordering?
----------------------------------------------------------------------------------------------------
- Cashflow4cast supports inventory tracking and reordering - The platform helps businesses stock the
right inventory at the right time - It enables businesses to make smarter decisions around inventory
and labor - Cashflow4cast allows businesses to stay ahead of invisible risks affecting their bottom
line


🔹 Test 2: Can I use this to file my taxes?
----------------------------------------------------------------------------------------------------
I'm sorry, I don't have that information based on the current documentation.


🔹 Test 3: How do I open a store in another state?
----------------------------------------------------------------------------------------------------
I'm sorry, I don't have that information based on the current documentation.



#### CLEAN WIDGETS

In [2]:
import json
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

notebook_path ="/content/drive/My Drive/LANGCHAIN/LC_011_RAG_CustServiceBot_PromptOptimzation.ipynb"

# Load the notebook JSON
with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = json.load(f)

# 1. Remove widgets from notebook-level metadata
if "widgets" in nb.get("metadata", {}):
    del nb["metadata"]["widgets"]
    print("✅ Removed notebook-level 'widgets' metadata.")

# 2. Remove widgets from each cell's metadata
for i, cell in enumerate(nb.get("cells", [])):
    if "metadata" in cell and "widgets" in cell["metadata"]:
        del cell["metadata"]["widgets"]
        print(f"✅ Removed 'widgets' from cell {i}")

# Save the cleaned notebook
with open(notebook_path, 'w', encoding='utf-8') as f:
    json.dump(nb, f, indent=2)

print("✅ Notebook deeply cleaned. Try uploading to GitHub again.")

Mounted at /content/drive
✅ Notebook deeply cleaned. Try uploading to GitHub again.
